# Black hole mergers project

Giacomo Menegatti, Dario Puggioni, Laura Schulze, Savina Tsichli

In [19]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


In [20]:
! ls -R


.:
AstroLab.ipynb	stable_MT_vs_CE

./stable_MT_vs_CE:
A0.5  A1  A3  A5

./stable_MT_vs_CE/A0.5:
MTCE_BBHs_0.0002.txt  MTCE_BBHs_0.0016.txt  MTCE_BBHs_0.008.txt
MTCE_BBHs_0.0004.txt  MTCE_BBHs_0.002.txt   MTCE_BBHs_0.012.txt
MTCE_BBHs_0.0008.txt  MTCE_BBHs_0.004.txt   MTCE_BBHs_0.016.txt
MTCE_BBHs_0.0012.txt  MTCE_BBHs_0.006.txt   MTCE_BBHs_0.02.txt

./stable_MT_vs_CE/A1:
MTCE_BBHs_0.0002.txt  MTCE_BBHs_0.0016.txt  MTCE_BBHs_0.008.txt
MTCE_BBHs_0.0004.txt  MTCE_BBHs_0.002.txt   MTCE_BBHs_0.012.txt
MTCE_BBHs_0.0008.txt  MTCE_BBHs_0.004.txt   MTCE_BBHs_0.016.txt
MTCE_BBHs_0.0012.txt  MTCE_BBHs_0.006.txt   MTCE_BBHs_0.02.txt

./stable_MT_vs_CE/A3:
MTCE_BBHs_0.0002.txt  MTCE_BBHs_0.0016.txt  MTCE_BBHs_0.008.txt
MTCE_BBHs_0.0004.txt  MTCE_BBHs_0.002.txt   MTCE_BBHs_0.012.txt
MTCE_BBHs_0.0008.txt  MTCE_BBHs_0.004.txt   MTCE_BBHs_0.016.txt
MTCE_BBHs_0.0012.txt  MTCE_BBHs_0.006.txt   MTCE_BBHs_0.02.txt

./stable_MT_vs_CE/A5:
MTCE_BBHs_0.0002.txt  MTCE_BBHs_0.0016.txt  MTCE_BBHs_0.008.txt
MTCE_B

In [21]:
alpha = [0.5, 1, 3, 5] # Alpha constants of the CE
metallicity = [2e-4, 4e-4, 8e-4, 1.2e-3, 1.6e-3, 2e-3, 4e-3, 6e-3, 8e-3, 1.2e-2, 1.6e-2, 2e-2]  #Metallicity value

a = alpha[0]
m = metallicity[-1]

data = pd.read_csv(f'stable_MT_vs_CE/A{a}/MTCE_BBHs_{m}.txt', sep=' ', nrows = 1, header=0) # All data
data.head()


,col.0:,MtotZAMS/Msun,Unnamed: 2,col.1:num,mergers
0,4.304914e+08,274,NaN,NaN,NaN


In [ ]:

data.head()
data.info()
